Title: identifying most popular country 

Author: Ivan Zheng

Date: 10/06

In [1]:
# Import and create a new SQLContext 
import pyspark
from pyspark.sql import SQLContext
from pyspark import SparkContext
sc = SparkContext()
sqlContext = SQLContext(sc)

NameError: name 'sc' is not defined

### Step 1: Transforming the Country Data to pyspark SQL formate

In [ ]:
# Read the country CSV file into an RDD.
country_lines = sc.textFile('data/country-list.csv')

In [ ]:
country_lines.take(5)

In [ ]:
# Convert each line into a pair of words
country_words = country_lines.map(lambda line : line.split(", "))

In [ ]:
country_words.take(5)

In [ ]:
# Convert each pair of words into a tuple
country_tuples = country_words.map(lambda x: (x[0], x[1]))

In [ ]:
country_tuples.take(5)

In [ ]:
# Create the DataFrame, look at schema and contents
countryDF = sqlContext.createDataFrame(country_tuples, ["country", "code"])
countryDF.printSchema()
countryDF.take(3)

###### Transformed the Country Date to pyspark SQL formate

### Step 2: Calculating the most popular country in tweet text

In [ ]:
# Read tweets CSV file into RDD of lines
tweet_lines = sc.textFile('data/tweet_output.csv')

In [ ]:
tweet_lines.take(3)

In [ ]:
tweet_lines.count() #before cleaning

In [ ]:
# Clean the data: some tweets are empty. Remove the empty tweets using filter() 
tweet_lines = tweet_lines.filter(lambda x: x is not ' ')

In [ ]:
tweet_lines.count() #before removing null

##### Perform WordCount on the cleaned tweet texts.

In [ ]:
tweet_words = tweet_lines.flatMap(lambda line : line.split(" "))

In [ ]:
tweet_tuples = tweet_words.map(lambda word : (word, 1))

In [ ]:
tweet_counts = tweet_tuples.reduceByKey(lambda a, b: (a + b))

In [ ]:
tweet_counts.take(5)

In [ ]:
# Create the DataFrame of tweet word counts
tweetDF = sqlContext.createDataFrame(tweet_counts, ["country", "counts"])
tweetDF.printSchema()
tweetDF.take(3)

In [ ]:
# Join the country and tweet data frames (on the appropriate column)
countryDF.printSchema()

In [ ]:
tweetDF.show(10)

In [ ]:
countryDF.show(10)

In [ ]:
mergeDF = countryDF.join(tweetDF, 'country')

In [ ]:
mergeDF.show(5)

In [ ]:
# Number of distinct countries mentioned
mergeDF.filter(mergeDF["counts"] > 0).count()

In [ ]:
#Number of countries mentioned in tweets.
from pyspark.sql.functions import sum
mergeDF.select(sum('counts')).show()

In [ ]:
# Table 1: top three countries and their counts.
from pyspark.sql.functions import desc
mergeDF.sort(mergeDF.counts.desc()).show(5)

In [ ]:
# Table 2: counts for Wales, Iceland, and Japan.
mergeDF.filter(mergeDF["country"] == 'Wales').show()

In [ ]:
mergeDF.filter(mergeDF["country"] == 'Kenya').show()

In [ ]:
mergeDF.filter(mergeDF["country"] == 'Netherlands').show()

In [ ]:
mergeDF.filter(mergeDF["country"] == 'Iceland').show()

In [ ]:
mergeDF.filter(mergeDF["country"] == 'Japan').show()

Summary, a short spark exercise using PySpark SQL and generate output using MonogDB.